In [2]:
import pandas as pd
import re

# ======================================================
# 1) 파일 불러오기
# ======================================================
file = "/content/지역별_안전 비상벨 정보.xlsx"
df = pd.read_excel(file)

# 컬럼명 자동 정리(양쪽 공백 제거)
df.columns = df.columns.str.strip()

# 주소 관련 컬럼명 설정
col_road = "소재지도로명주소"
col_addr = "소재지지번주소"
col_manage = "관리기관명"
col_year = "안전비상벨설치연도"

# ======================================================
# 2) 서울 25개 구 리스트
# ======================================================
seoul_gu_list = [
    "종로구","중구","용산구","성동구","광진구","동대문구","중랑구","성북구","강북구","도봉구",
    "노원구","은평구","서대문구","마포구","양천구","강서구","구로구","금천구","영등포구",
    "동작구","관악구","서초구","강남구","송파구","강동구"
]

# ======================================================
# 3) 주소에서 구 추출 함수
# ======================================================
def extract_gu(text):
    if pd.isna(text):
        return None
    pattern = r"(종로구|중구|용산구|성동구|광진구|동대문구|중랑구|성북구|강북구|도봉구|" \
              r"노원구|은평구|서대문구|마포구|양천구|강서구|구로구|금천구|영등포구|" \
              r"동작구|관악구|서초구|강남구|송파구|강동구)"
    m = re.search(pattern, str(text))
    return m.group(1) if m else None

# ======================================================
# 4) 최종 지역 추출 함수
# ======================================================
def get_final_region(row):
    r1 = extract_gu(row[col_road])
    if r1 in seoul_gu_list:
        return r1

    r2 = extract_gu(row[col_addr])
    if r2 in seoul_gu_list:
        return r2

    r3 = extract_gu(row[col_manage])
    if r3 in seoul_gu_list:
        return r3

    return "기타"

df["최종지역"] = df.apply(get_final_region, axis=1)

# ======================================================
# 5) 기타 목록 및 지역별 개수 출력
# ======================================================
기타값 = df[df["최종지역"]=="기타"]["최종지역"].unique()
print("----- 기타로 분류된 값 목록 -----")
print(기타값)

print("\n----- 지역별 개수 -----")
print(df["최종지역"].value_counts())

# ======================================================
# 6) 주소/관리기관명 전체에서 등장한 구를 저장 (이상치 검출용)
# ======================================================
def find_all_gu(text):
    if pd.isna(text):
        return []
    pattern = r"(종로구|중구|용산구|성동구|광진구|동대문구|중랑구|성북구|강북구|도봉구|" \
              r"노원구|은평구|서대문구|마포구|양천구|강서구|구로구|금천구|영등포구|" \
              r"동작구|관악구|서초구|강남구|송파구|강동구)"
    return re.findall(pattern, str(text))

df["주소내_모든_구"] = df.apply(
    lambda x: find_all_gu(f"{x[col_road]} {x[col_addr]} {x[col_manage]}"),
    axis=1
)

df_problem = df[df.apply(
    lambda x: x["최종지역"] not in x["주소내_모든_구"] and x["최종지역"] != "기타",
    axis=1
)]

print("\n 주소에서 추출한 구가 실제 문자열과 불일치한 행 수:", len(df_problem))
print(df_problem[[col_road, col_addr, col_manage, "주소내_모든_구", "최종지역"]].head(20))

# ======================================================
# 7) 설치연도 이상치 확인 및 제거
# ======================================================
# 2025년 초과 연도 데이터 추출 (제거 전 확인)
over_2025 = df[df[col_year] > 2025]
over_2025_info = over_2025[[col_year, "최종지역", col_road, col_addr, col_manage]]

print("\n----- 2025년 초과 설치 연도 데이터 (제거 대상) -----")
print(over_2025_info)
print(f"\n제거할 데이터 개수: {len(over_2025)}개")

if len(over_2025) > 0:
    print("\n제거되는 지역별 개수:")
    print(over_2025_info["최종지역"].value_counts())

# 이상치 제거: 2025년 이하 데이터만 남김
df_clean = df[(df[col_year] <= 2025) & df[col_year].notna()].copy()
df_clean["설치대수"] = 1

print(f"\n정상 데이터 개수: {len(df_clean)}개")

# ======================================================
# 8) 연도별 + 지역별 집계
# ======================================================
year_region = df_clean.groupby([col_year, "최종지역"])["설치대수"].sum().reset_index()
year_region = year_region.sort_values(by=[col_year, "최종지역"])

# 누적 설치대수 계산
year_region["누적설치대수"] = year_region.groupby("최종지역")["설치대수"].cumsum()

# ======================================================
# 9) CSV 생성
# ======================================================
year_region.to_csv("비상벨_년도별_지역별_누적설치대수.csv", index=False)
print("\n CSV 생성 완료: 비상벨_년도별_지역별_누적설치대수.csv")

/usr/local/lib/python3.12/dist-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


----- 기타로 분류된 값 목록 -----
[]

----- 지역별 개수 -----
최종지역
강남구     1713
중랑구     1520
성북구     1519
구로구     1442
용산구     1421
영등포구    1307
관악구     1279
마포구     1268
강서구     1156
광진구     1119
도봉구     1071
강동구     1037
동대문구    1013
성동구      900
송파구      881
양천구      787
중구       744
서대문구     723
금천구      709
동작구      627
은평구      611
종로구       99
노원구       64
서초구       52
강북구       22
Name: count, dtype: int64

 주소에서 추출한 구가 실제 문자열과 불일치한 행 수: 0
Empty DataFrame
Columns: [소재지도로명주소, 소재지지번주소, 관리기관명, 주소내_모든_구, 최종지역]
Index: []

----- 2025년 초과 설치 연도 데이터 (제거 대상) -----
       안전비상벨설치연도  최종지역               소재지도로명주소                소재지지번주소  \
16492       2031  영등포구     서울특별시 영등포구 대림동 742     서울특별시 영등포구 대림동 742   
16493       2032  영등포구     서울특별시 영등포구 대림동 690     서울특별시 영등포구 대림동 690   
16494       2033  영등포구     서울특별시 영등포구 대림동 715     서울특별시 영등포구 대림동 715   
16495       2034  영등포구    서울특별시 영등포구 도림동 23-1    서울특별시 영등포구 도림동 23-1   
16496       2035  영등포구  서울특별시 영등포구 문래동3가 97-1  서울특별시 영등포구 문래동3가 97-1   
16497       